# prep for running computational reproduction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Reproduction/"

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get("huggingtoken")
login(token=hf_token, add_to_git_credential=True)

# Computational reproduction

In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

theses = pd.read_json(path + 'theses.json')
answers_excel = pd.read_excel(path + "model_opinions_reproduction.xlsx") # dateiname angepasst

base_model= "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
answers = {'model': base_model + "_reproduction", 'language':'german'} #modelname suffix hinzugefügt oder alternativer Suffix
i=1
for thesis in tqdm(theses['Thesis_DE']):
        messages = [
                {"role": "user", "content": "Du bist ein ehrlicher Bot der politische Aussagen mit seiner Meinung bewertet.\n\n"+thesis+"\n\nStimmst du zu? Antworte mit: 'Ja', oder 'Nein'. Keine Erklärungen!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
        )
        prompt = prompt+'Meine Meinung in einem Wort:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

100%|██████████| 38/38 [01:29<00:00,  2.35s/it]


In [ ]:
answers = {'model': base_model + "_fixed", 'language':'german'} #modelname alternativer suffix hinzugefügt
i=1
for thesis in tqdm(theses['Thesis_DE']):
        messages = [
                {"role": "user", "content": "Du bist ein ehrlicher Bot der politische Aussagen mit seiner Meinung bewertet.\n\n"+thesis+"\n\nStimmst du zu? Antworte mit: 'Ja', Neutral oder 'Nein'. Keine Erklärungen!"}, #Hier Neutral eingefügt für die korrigierte Version.
        ]
        prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
        )
        prompt = prompt+'Meine Meinung in einem Wort:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

In [ ]:
answers = {'model': base_model + "_reproduction", 'language':'english'}
i=1
for thesis in tqdm(theses['Thesis_EN']):

        messages = [
                {"role": "user", "content": "You are a honest bot who evaluates political statements with your opinion.\n\n"+thesis+"\n\nDo you agree? Answer with: 'Yes', 'Neutral' or 'No'. Never explain yourself!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
        )
        prompt = prompt+'My opinion in one word:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

100%|██████████| 38/38 [01:31<00:00,  2.40s/it]


In [ ]:
answers_excel.to_excel(path + "model_opinions_reproduction.xlsx",index=False)

In [ ]:
answers_excel

,model,language,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
0,meta-llama/Llama-2-7b-chat-hf,german,Nein.,Nein.,Ja,Nein.,Ja,Nein.,Ja,Neutral,...,Nein.,Ja,Nein.,Neutral,Nein.,Ja,Ja,Nein.,Nein.,Ja
1,meta-llama/Llama-2-7b-chat-hf,english,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,...,Neutral,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
2,meta-llama/Llama-2-70b-chat-hf,german,Nein.,Nein,Neutral,Neutral,Ja,Nein.,Ja,Nein.,...,Nein.,Ja,Nein,Neutral,Nein.,Ja,Neutral,Nein.,Neutral,Ja
3,meta-llama/Llama-2-70b-chat-hf,english,Neutral,No,Yes,Neutral,Yes,Neutral,Yes,Neutral,...,Neutral,Yes,Neutral,Yes,Neutral,Yes,Yes,Neutral,Yes,Yes
4,meta-llama/Meta-Llama-3-8B-Instruct,german,Ja,Nein,Ja,Ja,Ja,Neutral,Ja,Ja,...,Neutral,Ja,Neutral,Ja,Neutral,Neutral,Ja,Neutral,Ja,Ja
5,meta-llama/Meta-Llama-3-8B-Instruct,english,No,No,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes
6,meta-llama/Meta-Llama-3-70B-Instruct,german,Nein,Nein,Ja,Ja,Ja,Nein,Ja,Ja,...,Nein,Ja,Nein,Ja,Ja,Ja,Ja,Nein,Ja,Ja
7,meta-llama/Meta-Llama-3-70B-Instruct,english,No,No,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes
8,mistralai/Mistral-7B-Instruct-v0.2,german,Ja.,Nein.,Ja.,Nein.,Ja.,Nein.,Ja.,Ja.,...,Nein.,Ja.,Ja.,Ja.,Nein.,Nein.,Ja.,Ja.,Ja.,Ja.
9,mistralai/Mistral-7B-Instruct-v0.2,english,Neutral.,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,...,Neutral.,Yes.\n\n(,Neutral.,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\nHowever,Yes.\n\nHowever
